## Connect with chinook database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Review the database for a comprehensive list of available tables and views

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Business concept #1
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce

## Create View with potential new artists 

In [4]:
%%sql
DROP VIEW IF EXISTS potential_artists;
CREATE VIEW potential_artists AS
    SELECT 'Regal' AS "Artist Name", 'Hip-Hop' AS "Genre"
        UNION
    SELECT 'Red Tone', 'Punk'
        UNION
    SELECT 'Meteor and the Girls', 'Pop'
        UNION
    SELECT 'Slim Jim Bites', 'Blues';

Done.
Done.


[]

In [5]:
%%sql
SELECT * FROM potential_artists;

Done.


Artist Name,Genre
Meteor and the Girls,Pop
Red Tone,Punk
Regal,Hip-Hop
Slim Jim Bites,Blues


# Analyse which genre sells the most tracks in the USA

In [6]:
%%sql
WITH invoice_genre AS
(
    SELECT
        i.invoice_id,
        t.track_id, 
        g.name AS Genre
    FROM 
        track AS t
            INNER JOIN invoice_line AS il ON il.track_id = t.track_id
            INNER JOIN invoice AS i ON il.invoice_id = i.invoice_id
            INNER JOIN genre AS g ON g.genre_id = t.genre_id
    WHERE i.billing_country = 'USA')
SELECT 
    Genre, 
    COUNT(*) AS sold_tracks,
    ROUND((COUNT(*) * 100.0) / (SELECT COUNT(*) FROM invoice_genre),2) AS sold_tracks_percentage
FROM invoice_genre 
    GROUP BY Genre 
    ORDER BY sold_tracks DESC;

Done.


Genre,sold_tracks,sold_tracks_percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


# Observations

Looking at the current best-selling songs based on genre, we should add Red Tone, Meteor and the Girls and Slim Jim Bites to the store

# Business Concept #2
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [7]:
%%sql
SELECT 
    e.employee_id, 
    e.hire_date,
    e.first_name || " " || e.last_name AS Employee_name,
    COUNT(i.invoice_id) AS Number_Of_Sales,
    ROUND(AVG(i.total),2) AS Average_Sale,
    ROUND(SUM(i.total),2) AS Total_Sales
FROM employee AS e 
    INNER JOIN customer AS c ON c.support_rep_id = e.employee_id
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
        GROUP BY e.employee_id
        ORDER BY Total_Sales DESC;

Done.


employee_id,hire_date,Employee_name,Number_Of_Sales,Average_Sale,Total_Sales
3,2017-04-01 00:00:00,Jane Peacock,212,8.17,1731.51
4,2017-05-03 00:00:00,Margaret Park,214,7.4,1584.0
5,2017-10-17 00:00:00,Steve Johnson,188,7.41,1393.92


# Observations
Jane Peacock has the highest sales and revenue.

Margaret Park has the most sales but slightly lower revenue.

Steve Johnson, hired later, performs well.

# Business Concept #3
We need to analyze the sales data for customer from each country, 
countries with only one customer has been grouped into "Other".
For each country, we include:

    total number of customers
    total value of sales
    average value of sales per customer
    average order value

In [37]:
%%sql
SELECT
    Country,
    SUM(customer_count) AS 'Total Number Of Customers',
    SUM(total_sales) AS 'Total Value Of Sales',
    ROUND(SUM(total_sales)  / SUM(customer_count),2) AS 'Average Value Of Sales Per Customer',
    ROUND(SUM(total_sales)  / SUM(invoice_count),2) AS 'Average Order Value'
FROM (
    SELECT
        CASE WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other' ELSE c.country END AS Country,
        COUNT(DISTINCT c.customer_id) AS customer_count,
        ROUND(SUM(i.total),2) AS total_sales,
        ROUND(COUNT(DISTINCT i.invoice_id),2) AS invoice_count
    FROM customer AS c
    INNER JOIN invoice AS i ON i.customer_id = c.customer_id
    GROUP BY c.country
) AS Subquery
GROUP BY Country
;

Done.


Country,Total Number Of Customers,Total Value Of Sales,Average Value Of Sales Per Customer,Average Order Value
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45
Portugal,2,185.13,92.56,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77


# Observations
While the total sales for "Other" are significant, the average value of sales per customer is comparatively lower, suggesting that these markets may require tailored strategies to increase customer spending.

The Czech Republic stands out with the highest average value of sales per customer at $136.62, indicating a potential opportunity for targeted marketing efforts or product offerings to further capitalize on this market's high customer spending.

While the USA has the highest total value of sales at $1040.49, its average value of sales per customer and average order value are both relatively lower compared to some other countries. This suggests potential for optimizing sales strategies to increase individual customer spending.

Overall, there are variations in customer behavior and spending patterns among different countries, emphasizing the importance of country-centric analysis for businesses to tailor their strategies effectively.

# Business Case #4

In response to management's consideration of changing the purchasing strategy, our objective is to analyze customer purchase patterns at the Chinook store. We aim to determine the percentage of purchases that consist of individual tracks versus whole albums. This analysis will provide valuable insights into customer behavior and help management make informed decisions regarding their purchasing strategy, potentially optimizing revenue while maintaining customer satisfaction.

In [52]:
%%sql
WITH album_purchases AS (
  SELECT
    il.invoice_id,
    CASE
      WHEN (
        SELECT track_id
        FROM track
        WHERE album_id = (
          SELECT album_id
          FROM track
          WHERE track_id = il.track_id
        )
      EXCEPT
        SELECT il2.track_id
        FROM invoice_line il2
        WHERE il2.invoice_id = il.invoice_id
      ) IS NULL
      AND (
        SELECT il2.track_id
        FROM invoice_line il2
        WHERE il2.invoice_id = il.invoice_id
      EXCEPT
        SELECT track_id
        FROM track
        WHERE album_id = (
          SELECT album_id
          FROM track
          WHERE track_id = il.track_id
        )
      ) IS NULL
      THEN "yes"
      ELSE "no"
    END AS album_purchase
  FROM invoice_line il
)

SELECT
  album_purchase,
  ROUND(COUNT(invoice_id),2) AS number_of_invoices,
  ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice),2) AS percent
FROM album_purchases
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,3281.0,5.34
yes,1476.0,2.4


# Observation
The majority of purchases (5.34% of invoices) consist of individual tracks rather than whole albums, indicating that customers tend to prefer selecting specific tracks rather than buying complete albums.

However, album purchases still account for a notable portion (2.4% of invoices) of total sales, suggesting that there is demand for purchasing complete albums as well. This data can help inform decisions regarding inventory and purchasing strategies to optimize revenue.